In [11]:
from PIL import Image
import numpy as np
import pickle
import os
from funkcije import *

image_path = "lena.bmp"

img = Image.open(image_path) #slika koju zelimo da kompresujemo

img = img.convert("L")

vr_piksela = list(img.getdata())

sekvenca_piksela = vr_piksela[:]

kompresovana_sekvenca = lzw_kompresija(sekvenca_piksela)

kompresovana_sekvenca_np = np.array(kompresovana_sekvenca, dtype=np.uint16)

with open('kompresovana_slika.pkl', 'wb') as f:   #wb - write binary, stavljamo naziv .pkl fajla
    pickle.dump(kompresovana_sekvenca_np, f, protocol=4)

dekompresovana_sekvenca = lzw_dekompresija(kompresovana_sekvenca)

new_img = Image.new("L", img.size)

new_img.putdata(dekompresovana_sekvenca)

new_img.show()

velicina_originalne = os.path.getsize(image_path) #slika koju zelimo da kompresujemo

velicina_kompresovane = os.path.getsize('kompresovana_slika.pkl') #.pkl fajl

CR=velicina_originalne/velicina_kompresovane

print(f"Odnos kompresije: {CR*100:.1f}%.")

SS=1-1/CR

print(f"Sačuvani prostor: {SS*100:.1f}%.")

suma=0

n=len(sekvenca_piksela)

for i in range(n):
    suma+=(sekvenca_piksela[i]-dekompresovana_sekvenca[i])**2

print(f"Srednja kvadratna greška: {suma/n}.")


Odnos kompresije: 375.2%.
Sačuvani prostor: 73.3%.
Srednja kvadratna greška: 0.0.
